In [1]:
import spynnaker8 as sim
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

In [2]:
 sim.setup(timestep=0.1, min_delay=1.0)

2021-07-08 07:18:48 INFO: Read cfg files: /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2021-07-08 07:18:48 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/common_model_binaries : /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries
2021-07-08 07:18:48 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2021-07-08 07:18:48 INFO: Setting time scale factor to 10.
2021-07-08 07:18:48 INFO: Setting machine time step to 100 micro-seconds.


['/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


0

In [3]:
model = sim.Izhikevich
a_tonic = 0.02
b_tonic = 0.2
c_tonic = -65.0
d_tonic = 6.0
v_init_tonic = -65.0
u_init_tonic = b_tonic * v_init_tonic
current_Pulse = 0
tau_ex = 1.7
tau_inh = 2.5
cell_params = {'a': a_tonic, 'b': b_tonic, 'c': c_tonic, 'd': d_tonic,
               'tau_syn_E': tau_ex, 'tau_syn_I': tau_inh,
               'i_offset': current_Pulse
               }

In [4]:
initial_values = {'v': v_init_tonic, 'u': u_init_tonic}

In [5]:
e_rate = 60
in_rate = 300

n_stim_test = 5
n_stim_pairing = 20
dur_stim = 20

pop_size = 40

ISI = 90.
start_test_pre_pairing = 200.
start_pairing = 1500.
start_test_post_pairing = 700.

simtime = (start_pairing + start_test_post_pairing
           + ISI * (n_stim_pairing + n_stim_test) + 550.)

Neuron Populations

In [6]:
IAddPre = []
IAddPost = []

pre_pop = sim.Population(pop_size, model(**cell_params),
                         initial_values=initial_values)
post_pop = sim.Population(pop_size, model(**cell_params),
                          initial_values=initial_values)

In [ ]:
for i in range(n_stim_test):
    IAddPre.append(sim.Population(
        pop_size, sim.SpikeSourcePoisson(
            rate=in_rate, start=start_test_pre_pairing + ISI * i,
            duration=dur_stim)))

In [8]:
for i in range(n_stim_pairing):
    IAddPre.append(sim.Population(
        pop_size, sim.SpikeSourcePoisson(
           rate=in_rate, start=start_pairing + ISI * i, duration=dur_stim)))
    IAddPost.append(sim.Population(
        pop_size, sim.SpikeSourcePoisson(
            rate=in_rate, start=start_pairing + ISI * i + 10,
            duration=dur_stim)))

In [9]:
for i in range(n_stim_test):
    IAddPre.append(sim.Population(
        pop_size, sim.SpikeSourcePoisson(
            rate=in_rate, start=(
                start_pairing + ISI * n_stim_pairing +
                start_test_post_pairing + ISI * i),
            duration=dur_stim)))

In [10]:
INoisePre = sim.Population(
    pop_size, sim.SpikeSourcePoisson(
        rate=e_rate, start=0, duration=simtime), label="expoisson")
INoisePost = sim.Population(
    pop_size, sim.SpikeSourcePoisson(
        rate=e_rate, start=0, duration=simtime), label="expoisson")

In [11]:
JEE = 5

ee_connector = sim.OneToOneConnector()

In [12]:
sim.Projection(
    INoisePre, pre_pop, ee_connector, receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=JEE))
sim.Projection(
    INoisePost, post_pop, ee_connector, receptor_type='excitatory',
    synapse_type=sim.StaticSynapse(weight=JEE))

projection from pre expoisson53 to post AbstractPopulationVertex_2 with connector OneToOneConnector()

In [13]:
for i in range(len(IAddPre)):
    sim.Projection(
        IAddPre[i], pre_pop, ee_connector, receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=JEE))
for i in range(len(IAddPost)):
    sim.Projection(
        IAddPost[i], post_pop, ee_connector, receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=JEE))

In [14]:
stdp_model = sim.STDPMechanism(
    timing_dependence=sim.SpikePairRule(
        tau_plus=20.0, tau_minus=20.0, A_plus=0.045, A_minus=0.045),
    weight_dependence=sim.AdditiveWeightDependence(w_min=0, w_max=10))

plastic_projection = sim.Projection(
    pre_pop, post_pop, sim.FixedProbabilityConnector(p_connect=0.5),
    synapse_type=stdp_model)

In [15]:
pre_pop.record(['v', 'spikes', 'gsyn_exc', 'gsyn_inh'])
post_pop.record(['v', 'spikes', 'gsyn_exc', 'gsyn_inh'])

2021-07-08 08:14:16 WARNING: You are trying to record the excitatory conductance from a model which does not use conductance input. You will receive current measurements instead.
2021-07-08 08:14:16 WARNING: You are trying to record the inhibitory conductance from a model which does not use conductance input. You will receive current measurements instead.


In [16]:
sim.run(simtime)

print("Weights:{}".format(plastic_projection.get('weight', 'list')))

2021-07-08 08:14:25 INFO: Starting execution process
2021-07-08 08:14:25 INFO: Simulating for 50000 0.1ms timesteps using a hardware timestep of 1000us
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2021-07-08 08:14:25 INFO: Time 0:00:00.023838 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2021-07-08 08:14:25 INFO: Time 0:00:00.021336 taken by DelaySupportAdder
2021-07-08 08:14:28 INFO: Time 0:00:03.136971 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-07-08 08:14:36 INFO: Time 0:00:08.196272 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-07

2021-07-08 08:14:58 INFO: Time 0:00:00.044202 taken by GraphBinaryGatherer
Running pair routing table compression on chip
|0%                          50%                         100%|

2021-07-08 08:15:01 INFO: Time 0:00:03.552067 taken by PairOnChipRouterCompression
Generating Router table report
|0%                          50%                         100%|
2021-07-08 08:15:01 INFO: Time 0:00:00.011811 taken by unCompressedRoutingTableReports
loading fixed routes
|0%                          50%                         100%|
2021-07-08 08:15:01 INFO: Time 0:00:00.040606 taken by LoadFixedRoutes
Executing data specifications and loading data for system vertices using Java
|0%                          50%                         100%|
2021-07-08 08:15:03 INFO: Time 0:00:01.202286 taken by HostExecuteSystemDataSpecification
Loading system executables onto the machine
|0%                          50%                         100%|
2021-07-08 08:15:08 INFO: Time 0:00:05.756568 taken by Lo

2021-07-08 08:16:30 INFO: Time 0:00:00.104946 taken by PlacementsProvenanceGatherer
2021-07-08 08:16:30 INFO: Time 0:00:00.000992 taken by RedundantPacketCountReport
Getting Router Provenance
|0%                          50%                         100%|
2021-07-08 08:16:30 INFO: Time 0:00:00.052726 taken by RouterProvenanceGatherer
Getting profile data
|0%                          50%                         100%|
2021-07-08 08:16:30 INFO: Time 0:00:00.093391 taken by ProfileDataGatherer
Getting synaptic data between AbstractPopulationVertex_1 and AbstractPopulationVertex_2
|0%                          50%                         100%|


Weights:[( 0,  0, 4.90625 ) ( 0,  1, 3.53125 ) ( 0,  2, 4.34375 )
 ( 0,  3, 4.5625  ) ( 0,  4, 3.09375 ) ( 0,  5, 3.21875 )
 ( 0,  6, 2.5     ) ( 0,  7, 2.90625 ) ( 0,  9, 2.828125)
 ( 0, 10, 3.09375 ) ( 0, 11, 3.421875) ( 0, 14, 4.09375 )
 ( 0, 16, 3.6875  ) ( 0, 17, 3.84375 ) ( 0, 20, 2.796875)
 ( 0, 23, 2.328125) ( 0, 25, 3.90625 ) ( 0, 26, 3.453125)
 ( 0, 28, 3.5     ) ( 0, 29, 2.203125) ( 0, 31, 2.609375)
 ( 0, 34, 2.      ) ( 0, 35, 2.921875) ( 0, 36, 3.25    )
 ( 0, 38, 3.796875) ( 0, 39, 4.359375) ( 1,  1, 2.703125)
 ( 1,  2, 2.25    ) ( 1,  4, 2.09375 ) ( 1,  5, 2.328125)
 ( 1,  9, 3.      ) ( 1, 10, 3.90625 ) ( 1, 12, 1.171875)
 ( 1, 13, 1.984375) ( 1, 14, 2.65625 ) ( 1, 19, 0.8125  )
 ( 1, 24, 4.234375) ( 1, 26, 3.125   ) ( 1, 27, 2.359375)
 ( 1, 33, 2.609375) ( 1, 34, 1.890625) ( 1, 35, 2.421875)
 ( 1, 36, 3.09375 ) ( 1, 38, 1.734375) ( 1, 39, 2.9375  )
 ( 2,  0, 1.265625) ( 2,  1, 0.      ) ( 2,  3, 0.390625)
 ( 2,  4, 1.      ) ( 2,  5, 1.21875 ) ( 2,  6, 0.      )
 ( 2, 

In [17]:
pre_spikes = pre_pop.get_data('spikes')
post_spikes = post_pop.get_data('spikes')
pre_gsyn_e = pre_pop.get_data("gsyn_exc")
post_gsyn_e = post_pop.get_data("gsyn_exc")
pre_gsyn_i = pre_pop.get_data("gsyn_inh")
post_gsyn_i = post_pop.get_data("gsyn_inh")

Getting spikes for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting spikes for AbstractPopulationVertex_2
|0%                          50%                         100%|
Getting gsyn_exc for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_exc for AbstractPopulationVertex_2
|0%                          50%                         100%|
Getting gsyn_inh for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting gsyn_inh for AbstractPopulationVertex_2
|0%                          50%                         100%|


In [18]:
Figure(
    Panel(pre_gsyn_e.segments[0].filter(name="gsyn_exc")[0],
          ylabel="pre_gsyn_e", yticks=True, markersize=0.2, xlim=(0, simtime),
          legend=False),
    Panel(post_gsyn_e.segments[0].filter(name="gsyn_exc")[0],
          ylabel="post_gsyn_e", yticks=True, xticks=True, markersize=0.2,
          xlim=(0, simtime), legend=False),
    Panel(pre_gsyn_i.segments[0].filter(name="gsyn_inh")[0],
          ylabel="pre_gsyn_i", yticks=True, markersize=0.2, xlim=(0, simtime),
          legend=False),
    Panel(post_gsyn_i.segments[0].filter(name="gsyn_inh")[0],
          ylabel="post_gsyn_i", yticks=True, xticks=True, markersize=0.2,
          xlim=(0, simtime), legend=False),
    # raster plot of the pre- and post-synaptic neuron spike times
    Panel(pre_spikes.segments[0].spiketrains,
          yticks=True, markersize=2.0, xlim=(0, simtime), color="red",
          ylabel="Pre-Population"),
    Panel(post_spikes.segments[0].spiketrains,
          yticks=True, xticks=True, markersize=2.0,
          xlim=(0, simtime), color="blue", ylabel="Post-Population"),
    title="Excitatory synaptic conductance (mS) and spike raster plots",
    annotations="Simulated with {}".format(sim.name())
)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/bbpnrsoa/sPyNNakerGit/lib/python3.8/site-packages/pyNN/utility/plotting.py:106: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  ax.plot(spiketrain,
/home/bbpnrsoa/sPyNNakerGit/lib/python3.8/site-packages/pyNN/utility/plotting.py:106: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  ax.plot(spiketrain,
/home/bbpnrsoa/sPyNNakerGit/lib/python3.8/site-packages/pyNN/utility/plotting.py:106: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  ax.plot(spiketrain,
/home/bbpnrsoa/sPyNNakerGit/lib/python3.8/site-packages/pyNN/utility/plotting.py:106: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The k

In [19]:
plt.close()

In [20]:
sim.end()